In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/marvel.csv', index_col=0)

In [3]:
df.head(3)

,comicID,characterID,name,title,year,Alignment_x,Intelligence_x,Strength,Speed,Durability_x,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,16232,1009220,Captain America,Cap Transport (2005) #12,2005.0,good,63.0,19.0,35.0,56.0,...,False,False,False,False,False,False,False,False,False,False
1,16232,1010740,Winter Soldier,Cap Transport (2005) #12,2005.0,good,56.0,32.0,35.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,16248,1009220,Captain America,Cap Transport (2005) #9,2005.0,good,63.0,19.0,35.0,56.0,...,False,False,False,False,False,False,False,False,False,False


In [4]:
#  exitem dois Ids para a Jean Grey, estamos padronizando esse Id
df = df.replace(1009496, 1009327)

In [5]:
relations_to_comics = {}

comics = df['comicID'].unique()

for i in comics:
    relations_to_comics[i] = []

In [6]:
chars = df['characterID'].unique()

for i in range(len(df)):
    comicID, charID = df.iloc[i, 0:2]
    if charID not in relations_to_comics[comicID]:
        relations_to_comics[comicID].append(charID)

In [7]:
to_remove = []
for i in relations_to_comics:
    if len(relations_to_comics[i]) <= 1:
        to_remove.append(i)

In [8]:
for i in to_remove:
    del relations_to_comics[i]
# relations_to_comics

In [9]:
relations_to_comics
temp = []
for n in relations_to_comics:
    temp.append(relations_to_comics[n])

In [10]:
import itertools
connections = []

for i in temp: 
    for j in list(itertools.combinations(i, 2)): 
        if j not in connections:
            connections.append(j)

In [11]:
result_df = df.drop_duplicates(subset=['name'], keep='first')
# result_df

In [12]:
names = list(result_df['name'])
ids = list(result_df['characterID'])

len(ids), len(names)

(1169, 1169)

In [13]:
from unidecode import unidecode
with open('marvel.gml', 'w') as file:

        # Primeira linha, que abre os colchetes da rede.
        file.write('graph [\n')

        # Segunda linha, que indica se a rede é dirigida (1) ou não (0).
        file.write('  directed 0\n')

        # Colchetes de cada nó. Você sempre precisa colocar um id (inteiro
        # ou string) e depois pode colocar os atributos adicionais que
        # quiser, contanto que sejam inteiros, floats ou strings. Se forem
        # strings, não esqueça as aspas duplas (isso vale para o id também).
        # Não esqueça também da indentação. Ela não é necessária mas ajuda
        # a deixar mais legível.
        #
        # O módulo unidecode converte todo caractere não-ASCII para o
        # caractere ASCII mais próximo. Isso é necessário porque a
        # especificação do formato gml exige que ele seja ASCII.
        for i in range(len(names)):
            file.write('  node [\n')
            file.write('    id {}\n'.format(ids[i]))
            file.write('    name "{}"\n'.format(unidecode(names[i])))
            file.write('  ]\n')

        # Colchetes de cada aresta. Você sempre precisa colocar um source
        # e um target (ids de nós) e depois pode colocar os atributos
        # adicionais que quiser, contanto que sejam inteiros, floats ou
        # strings. Se forem strings, não esqueça as aspas duplas (isso
        # vale para o source e o target também). Não esqueça também da
        # indentação. Ela não é necessária mas ajuda a deixar mais legível.
        for (n, m) in connections:
            file.write('  edge [\n')
            file.write('    source {}\n'.format(n))
            file.write('    target {}\n'.format(m))
            file.write('  ]\n')

        # Última linha, que fecha os colchetes da rede.
        file.write(']\n')